In [ ]:
# EP2_Augmentation
### EP2 MAC0417 / MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
# DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
# #
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
# DESONESTIDADE ACADÊMICA. #
# #
# #
# Nome : Jose Vitor Pereira Garzon
# NUSP : 9267071
# Turma: MAC0417-145-2024
# Prof.: Ronaldo Fumio Hashimoto
# Monitor: Marcos Freitas Parra
##################################################################

# Criando OriginalGrayDataset Contorno, Area, Maior Area, Imagens Escolhidas

In [1]:
import os
import json
from PIL import Image

diretorio_imagens_escolhidas = r'C:\Users\josef\Desktop\EP3\Contorno_Tracado_Selecionadas' # Pasta onde colocou as imagens selecionadas

def obter_fundo(nome_arquivo):
    if 'Fundo Branco' in nome_arquivo:
        return 'Branco'
    elif 'Fundo Preto' in nome_arquivo:
        return 'Preto'
    elif 'Fundo Rosa' in nome_arquivo:
        return 'Rosa'
    elif 'Fundo Verde' in nome_arquivo:
        return 'Verde'
    else:
        return 'Desconhecido'

def obter_luminosidade(nome_arquivo):
    if 'Dia_Claro' in nome_arquivo:
        return 'Dia_Claro'
    elif 'Dia_Escuro' in nome_arquivo:
        return 'Dia_Escuro'
    elif 'Noite_Claro' in nome_arquivo:
        return 'Noite_Claro'
    elif 'Noite_Escuro' in nome_arquivo:
        return 'Noite_Escuro'
    else:
        return 'Desconhecido'

def obter_classe(nome_arquivo):
    if 'Classe A' in nome_arquivo:
        return ["Tampa_Celular", "Escova_de_Dente", "Carregador", "Tomada"]
    elif 'Classe B' in nome_arquivo:
        return ["Coracao", "Quadrado_Preto", "Moeda_1"]
    elif 'Classe C' in nome_arquivo:
        return ["Moeda_Prateada", "Pedra_Transparente", "Pedra_Esverdeada", "Pedra_Amarela"]
    else:
        return 'Desconhecido'

def obter_informacoes_imagens(diretorio):
    informacoes_imagens = {}
    for filename in os.listdir(diretorio):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            try:
                imagem_path = os.path.join(diretorio, filename)
                with Image.open(imagem_path) as img:
                    informacoes_imagens[filename] = {
                        'Fundo': obter_fundo(filename),
                        'Luminosidade': obter_luminosidade(filename),
                        'Classes': obter_classe(filename),
                        'Processamento': 'Escala_Cinza',
                        'Caminho': imagem_path,
                        'Tipo de Binarizacao': 'Threshold'
                    }
            except Exception as e:
                print(f"Erro ao processar imagem {filename}: {e}")
    return informacoes_imagens

# Chamada da função para obter as informações das imagens
imagens_info = obter_informacoes_imagens(diretorio_imagens_escolhidas)

# Caminho para salvar o arquivo JSON
diretorio_atual = os.getcwd()
saida_json = os.path.join(diretorio_atual, 'Metadados_Threshold_Contornos_Escolhidas.json')

# Salvando as informações em um arquivo JSON
with open(saida_json, 'w') as outfile:
    json.dump(imagens_info, outfile, indent=2)

print(f'Arquivo JSON salvo com sucesso em: {saida_json}')


Arquivo JSON salvo com sucesso em: C:\Users\josef\Desktop\EP3\Metadados_Threshold_Contornos_Escolhidas.json


### Extração Caracteristicas OriginalGrayDataset Contorno, Area, Maior Area, Imagens Escolhidas

In [ ]:
from skimage.measure import label, regionprops
import os
import json
import numpy as np
import cv2
import pandas as pd

def Extracao(categoria, nome_arquivo, imagem_original):

    if len(imagem_original.shape) > 2:
        imagem_cinza = cv2.cvtColor(imagem_original, cv2.COLOR_BGR2GRAY)
    else:
        imagem_cinza = imagem_original.copy()

    # Rotular regiões conectadas na imagem em escala de cinza
    label_img = label(imagem_cinza, connectivity=2)
    
    # Extrair propriedades das regiões rotuladas
    props = regionprops(label_img)
    
    # Ordenar os contornos por área em ordem decrescente
    areas = [prop.area for prop in props]
    props = [props[i] for i in np.argsort(areas)[::-1]]
    
    # Atribuir identificadores aos objetos
    for i, prop in enumerate(props):
        prop.obj_id = i
    
    return props

def obter_classe(nome_arquivo):
    if 'Classe A' in nome_arquivo:
        return ["Tampa_Celular", "Escova_de_Dente", "Carregador", "Tomada"]
    elif 'Classe B' in nome_arquivo:
        return ["Coracao", "Quadrado_Preto", "Moeda_1"]
    elif 'Classe C' in nome_arquivo:
        return ["Moeda_Prateada", "Pedra_Transparente", "Pedra_Esverdeada", "Pedra_Amarela"]
    else:
        return 'Desconhecido'

# Caminho para o arquivo JSON contendo os dados das imagens
diretorio_atual = os.getcwd()
caminho_arquivo_json = os.path.join(diretorio_atual, "Metadados_Threshold_Contornos_Escolhidas.json")

# Lista para armazenar as linhas da tabela
tabela_dados = []

# Carregar dados do arquivo JSON
with open(caminho_arquivo_json, 'r') as arquivo:
    Metadados = json.load(arquivo)

# Percorre os metadados e converte cada imagem para escala de cinza
for nome_imagem, dados_imagem in Metadados.items():
    categoria = f"{dados_imagem['Fundo']}_{dados_imagem['Luminosidade']}"
    
    try:
        caminho_completo_imagem = dados_imagem['Caminho']  # Usar o caminho fornecido no JSON
        imagem_original = cv2.imread(caminho_completo_imagem)
        
        # Chame a função Extracao para processar a imagem
        caracteristicas = Extracao(categoria, caminho_completo_imagem, imagem_original)

        # Preencher os dados da tabela com as características de cada região
        for prop in caracteristicas:
            obj_id = obter_classe(nome_imagem)
            classe = obter_classe(nome_imagem)
            n_contorno = len(prop.coords)
            area = prop.area
            diameter = prop.equivalent_diameter
            minor_axis = prop.minor_axis_length
            imagem = os.path.splitext(os.path.basename(caminho_completo_imagem))[0]
            
            # Adicionar os dados como uma linha na tabela
            tabela_dados.append([obj_id, classe, n_contorno, area, diameter, minor_axis, imagem])
        
    except Exception as e:
        print(f"Erro ao processar {nome_imagem}: {str(e)}")

# Criar um DataFrame pandas com os dados coletados
df = pd.DataFrame(tabela_dados, columns=['Objeto', 'Classe', 'n_contorno', 'Área', 'Diâmetro', 'Minor', 'Imagem'])

# Exibir a tabela final
print(df)
